In [1]:
"""Colocar searchtags"""

'Colocar searchtags'

In [2]:
"""Para setar localização:



from django.contrib.gis.geos import Point, GeometryCollection

geometry=GeometryCollection([Point()])


"""

'Para setar localização:\n\n\n\nfrom django.contrib.gis.geos import Point, GeometryCollection\n\ngeometry=GeometryCollection([Point()])\n\n\n'

In [3]:
"""
record=OcurrenceRecord()
record.save()

service_order=ServiceOrder(company=company, description=texto_excel,kind="LAND",process_type="1",)
service_order.save()

record.service_orders.add(service_order)
service_order.responsibles.add(User.objects.get(...))

service_order_action=ServiceOrderAction(name=fazer loop em opções da company.metadata, criando os itens abaixo da escolha do excel
                    (incluindo), em ordem, estimated_end_date=date.today()+timedelta(week=valor no loop),
                    created_by=engie, service_order=service_order)
service_order_action.save()

procedure=Procedure(action=service_order_action,responsible=User.objects.get(...),to_do=texto_excel,
                    created_by=engie)

"""

'\nrecord=OcurrenceRecord()\nrecord.save()\n\nservice_order=ServiceOrder(company=company, description=texto_excel,kind="LAND",process_type="1",)\nservice_order.save()\n\nrecord.service_orders.add(service_order)\nservice_order.responsibles.add(User.objects.get(...))\n\nservice_order_action=ServiceOrderAction(name=fazer loop em opções da company.metadata, criando os itens abaixo da escolha do excel\n                    (incluindo), em ordem, estimated_end_date=date.today()+timedelta(week=valor no loop),\n                    created_by=engie, service_order=service_order)\nservice_order_action.save()\n\nprocedure=Procedure(action=service_order_action,responsible=User.objects.get(...),to_do=texto_excel,\n                    created_by=engie)\n\n'

In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [2]:
Company.objects.all()

<QuerySet [<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>, <Company: c85df77c-3e92-4f60-b9c1-de5866400d2d: GSO - UHE-CANA BRAVA>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: 37b4c80d-3f2b-4032-8cac-6ba1560d3144: GSO - UHE-SALTO SANTIAGO>, <Company: 4a5e0119-fb85-40fd-87ab-f201a2aa0fde: GSB - UHE-MACHADINHO>, <Company: 7fe7b473-6d9c-49d6-a9ca-74f3424d3211: GSB - UHE-SALTO OSORIO>, <Company: ed749424-27e4-4342-82cd-f924de71830b: Limbo - UTPS>, <Company: 1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA>, <Company: b75339ff-e91f-4e75-a082-5d7e23a4d049: GSB - UHE-SALTO SANTIAGO>, <Company: f0bdd035-6af9-401c-9233-26c5cdd808b0: GSO - UHE-JAGUARA>, <Company: 62ad80d9-b785-4037-933a-e23238728220: GSO - UHE-MIRANDA>, <Company: d25d0eb2-61a8-4646-8e0e-31c8e68cf216: TESTE>, <Company: 5f0ef84c-9985-4519-95de-e87fb4fcea21: GSO - PCH-JOSÉ GELÁZIO DA ROCHA>, <Company: 1b

In [3]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta, date, datetime
import random
from copy import deepcopy
import math
import locale
from tqdm.notebook import tqdm
import pytz
from django.contrib.gis.geos import Point, GeometryCollection
import subprocess
import sys
from django.contrib.admin.utils import flatten
from helpers.strings import get_obj_from_path, keys_to_snake_case, to_snake_case, UF_CODE
from uuid import uuid4
from django.contrib.gis.geos import Point

In [4]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [5]:
def get_inner_value(field, inner_field, occ, value):
    values = occ.form_fields['fields']
#     value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['apiName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == inner_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [6]:
filename='mirandaReg'

In [7]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
# sheetname
ws = wb[sheetname]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

In [9]:
values[0]

{'N. da Invasão': 'UHMI-001AD',
 'Região': 'Fazenda Boa Vista',
 'Fuso': 22,
 'Coordenada X': 813573,
 'Coordenada Y': 7907508,
 'Município': 'Indianópolis/MG',
 'Status': 'Registro em elaboração',
 'Equipe': 'LR Ambiental',
 'Data do registro': datetime.datetime(2020, 6, 9, 0, 0),
 'Classe': 'Uso e Ocupação Irregular',
 'Identificação do local_1': 'Área de inundação',
 'Identificação do local_2': 'Área de inundação',
 'Identificação do local_3': None,
 'Identificação do local_4': None,
 'Identificação do local_5': None,
 'Identificação do local_6': None,
 'Identificação do local_7': None,
 'Identificação do local_8': None,
 'Constatação_1': 'Ocupação/utilização não autorizada',
 'Constatação_2': 'Ocupação/utilização não autorizada',
 'Constatação_3': None,
 'Constatação_4': None,
 'Constatação_5': None,
 'Constatação_6': None,
 'Contatação_7': None,
 'Constatação_8': None,
 'Constatação (detalhe)_1': 'Trapiche fixo',
 'Constatação (detalhe)_2': 'Trapiche fixo',
 'Constatação (detalhe)

In [26]:
company = Company.objects.get(name='GES - UHE-MIRANDA')
# user = User.objects.get(first_name__icontains="dhonata")
user = User.objects.get(username="lucas.reis")

In [16]:
occ=OccurrenceType.objects.get(uuid='faa62cbd-c2ec-49fc-bc4a-9b1ab9edcca4')
user=User.objects.get(username='lucas.reis')
firm=Firm.objects.get(company=company,name='LR Ambiental')
tag1=SearchTag.objects.get(company = company, name = "Ocorrência")
tag3=SearchTag.objects.get(company = company, name = "Patrimonial")
tag4=SearchTag.objects.get(company = company, name = "Uso e ocupação irregular")
actions=get_obj_from_path(company.metadata, "land_action_names")

array_name='irregularities'

In [14]:
tag1.name

'Ocorrência'

In [15]:
tag3.name

'Patrimonial'

In [16]:
tag4.name

'Uso e ocupação irregular'

In [12]:
def get_action_list(actions, value):
    value_index = 0
    for index, a in enumerate(actions):
        if a['name'] == value:
            value_index=index
            break

    new_actions=actions[value_index:]
    
    return new_actions

In [18]:
def transform_coordinates(x, y, srid=31982):

    pnt = Point(x, y, srid=srid)
    pnt.transform(4326)

    lng = pnt.coords[0]
    lat = pnt.coords[1]
    return Point(lng, lat)

In [19]:
def get_uf(uf_code):
    if type(uf_code) != 'str':
        uf_code=str(uf_code)
    
    uf=[{'id': '12', 'name': 'AC'}, {'id': '27', 'name': 'AL' }, {'id': '13', 'name': 'AM'}, {'id': '16', 'name': 'AP'}, 
        {'id': '29', 'name': 'BA'}, {'id': '23', 'name': 'CE' }, {'id': '53', 'name': 'DF'}, {'id': '32', 'name': 'ES'}, 
        {'id': '52', 'name': 'GO'}, {'id': '21', 'name': 'MA' }, {'id': '31', 'name': 'MG'}, {'id': '50', 'name': 'MS'}, 
        {'id': '51', 'name': 'MT'}, {'id': '15', 'name': 'PA' }, {'id': '25', 'name': 'PB'}, {'id': '26', 'name': 'PE'}, 
        {'id': '22', 'name': 'PI'}, {'id': '41', 'name': 'PR' }, {'id': '33', 'name': 'RJ'}, {'id': '24', 'name': 'RN'},
        {'id': '11', 'name': 'RO'}, {'id': '14', 'name': 'RR' }, {'id': '43', 'name': 'RS'}, {'id': '42', 'name': 'SC'},
        {'id': '28', 'name': 'SE'}, {'id': '35', 'name': 'SP' }, {'id': '17', 'name': 'TO'}]
    uf_translation={item['name']: item['id'] for item in uf}
#     print(uf_translation)    
    
    return uf_translation[uf_code]

In [20]:
def get_involved_parts(a):
    return_dict = list(filter(None, [{
        'involved_parts': '1',
        'name': a['Nome do infrator_1'],
        'rg': True if a.get('RG') is not None else False,
        'description_rg': str(a.get('RG')).strip() if a.get('RG') is not None else None,
        'cpf': True if a.get('CPF / CNPJ') is not None else False,
        'description_cpf': str(a.get('CPF / CNPJ')).strip() if a.get('CPF / CNPJ') is not None else None,
        'phone': True if a.get('Telefone_1') is not None else False,
        'description_phone': str(a.get('Telefone_1')).strip() if a.get('Telefone_1') is not None else None,
        'email': True if a.get('E-mail') is not None else False,
        'description_email': a.get('E-mail').strip() if a.get('E-mail') is not None else None,
        'address': a.get('Logradouro').strip() if a.get('Logradouro') is not None else None,
        'address_number': str(a.get('Número')) if a.get('Número') is not None else None,
        'district': a.get('Bairro').strip() if a.get('Bairro') is not None else None,
        'address_additional': a.get('Complemento').strip() if a.get('Complemento') is not None else None,
#         'address_additional': a['Complemento'] if a['Complemento'] is not None else None,
        'uf': int(get_uf(a.get('UF').strip())) if a.get('UF') is not None else None,
        'city': a.get('Cidade').strip() if a.get('Cidade') is not None else None,
        'cep': str(a.get('CEP').strip()) if a.get('CEP') is not None else None
    },{
        'involved_parts': '2',
        'name': a['Nome do infrator_2'],
        'rg': True if a.get('RG') is not None else False,
        'description_rg': str(a.get('RG')).strip() if a.get('RG') is not None else None,
        'cpf': True if a.get('CPF / CNPJ') is not None else False,
        'description_cpf': str(a.get('CPF / CNPJ')).strip() if a.get('CPF / CNPJ') is not None else None,
        'phone': True if a.get('Telefone_2') is not None else False,
        'description_phone': str(a.get('Telefone_2')).strip() if a.get('Telefone_2') is not None else None,
        'email': True if a.get('E-mail') is not None else False,
        'description_email': a.get('E-mail').strip() if a.get('E-mail') is not None else None,
        'address': a.get('Logradouro').strip() if a.get('Logradouro') is not None else None,
        'address_number': str(a.get('Número')) if a.get('Número') is not None else None,
        'district': a.get('Bairro').strip() if a.get('Bairro') is not None else None,
        'address_additional': a.get('Complemento').strip() if a.get('Complemento') is not None else None,
#         'address_additional': a['Complemento'] if a['Complemento'] is not None else None,
        'uf': int(get_uf(a.get('UF').strip())) if a.get('UF') is not None else None,
        'city': a.get('Cidade').strip() if a.get('Cidade') is not None else None,
        'cep': str(a.get('CEP').strip()) if a.get('CEP') is not None else None
    }]))
    
    return return_dict

In [21]:
for a in tqdm(values):
    a['obs'] = list(filter(None, [{'observation_description': a['Observações_1']} if a['Observações_1'] else None,
                                 {'observation_description': a['Observações_2']} if a['Observações_2'] else None,
                                 {'observation_description': a['Observações_3']} if a['Observações_3'] else None,
                                 {'observation_description': a['Observações_4']} if a['Observações_4'] else None,
                                 {'observation_description': a['Observações_5']} if a['Observações_5'] else None,]))

  0%|          | 0/1025 [00:00<?, ?it/s]

In [23]:
for a in tqdm(values):
    a['irreg'] = list(filter(None,[{'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_1']) if a['Identificação do local_1'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_1']) if a['Constatação_1'] is not None else None,
                   'place_identification': get_inner_value(array_name, 'Constatação (detalhe)', occ, a['Constatação (detalhe)_1'].strip()) if a['Constatação (detalhe)_1'] is not None else None,
                   'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_1']) if a['Irregularidades_1'] is not None else None,
                   'other_description': a['Outros - Descrição_1'] if a['Outros - Descrição_1'] is not None else None,
                   'display_length': True if a['Comprimento (m)_1'] is not None else False,
                   'length': a['Comprimento (m)_1'] if a['Comprimento (m)_1'] is not None else None,
                   'display_width': True if a['Largura (m)_1'] is not None else False,
                   'width': a['Largura (m)_1'] if a['Largura (m)_1'] is not None else None,
                   'display_height': True if a['Altura (m)_1'] is not None else False,
                   'height': a['Comprimento (m)_1'] if a['Altura (m)_1'] is not None else None,
                   'display_area': True if a['Área (m2)_1'] is not None else False,
                   'area': a['Área (m2)_1'] if a['Área (m2)_1'] is not None else None,
                   'display_amount': True if a['Quantidade (un)_1'] is not None else False,
                   'amount': a['Quantidade (un)_1'] if a['Quantidade (un)_1'] is not None else None,
                   'display_volume': True if a['Volume (m3)_1'] is not None else False,
                   'volume': a['Volume (m3)_1'] if a['Volume (m3)_1'] is not None else None,
                   'display_deepen': True if a['Profundidade (m)_1'] is not None else False,
                   'deepen': a['Profundidade (m)_1'] if a['Profundidade (m)_1'] is not None else None,
                   'display_perimeter': True if a['Perímetro (m)_1'] is not None else False,
                   'perimeter': a['Diâmetro (m)_1'] if a['Perímetro (m)_1'] is not None else None,
                   'display_diameter': True if a['Diâmetro (m)_1'] is not None else False,
                   'diameter': a['Diâmetro (m)_1'] if a['Diâmetro (m)_1'] is not None else None,
                   'display_quote': True if a['Cota (m)_1'] is not None else False,
                   'quote': a['Cota (m)_1'] if a['Cota (m)_1'] is not None else None,
                   'display_kg': True if a['Peso (kg)_1'] is not None else False,
                   'weight': a['Peso (kg)_1'] if a['Peso (kg)_1'] is not None else None                  
                  },
                 {'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_2']) if a['Identificação do local_2'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_2']) if a['Constatação_2'] is not None else None,
                   'place_identification': get_inner_value(array_name, 'Constatação (detalhe)', occ, a['Constatação (detalhe)_2']) if a['Constatação (detalhe)_2'] is not None else None,
                   'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_2']) if a['Irregularidades_2'] is not None else None,
                   'other_description': a['Outros - Descrição_2'] if a['Outros - Descrição_2'] is not None else None,
                   'display_length': True if a['Comprimento (m)_2'] is not None else False,
                   'length': a['Comprimento (m)_2'] if a['Comprimento (m)_2'] is not None else None,
                   'display_width': True if a['Largura (m)_2'] is not None else False,
                   'width': a['Largura (m)_2'] if a['Largura (m)_2'] is not None else None,
                   'display_amount': True if a['Quantidade (un)_2'] is not None else False,
                   'amount': a['Quantidade (un)_2'] if a['Quantidade (un)_2'] is not None else None                 
                 } if a['Identificação do local_2'] is not None else None,
                 {'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_3']) if a['Identificação do local_3'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_3']) if a['Constatação_3'] is not None else None,
                   'place_identification': get_inner_value(array_name, 'Constatação (detalhe)', occ, a['Constatação (detalhe)_3']) if a['Constatação (detalhe)_3'] is not None else None,
                   'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_3']) if a['Irregularidades_3'] is not None else None,
                   'other_description': a['Outros - Descrição_3'] if a['Outros - Descrição_3'] is not None else None,
                   'display_length': True if a['Comprimento (m)_3'] is not None else False,
                   'length': a['Comprimento (m)_3'] if a['Comprimento (m)_3'] is not None else None,
                   'display_width': True if a['Largura (m)_3'] is not None else False,
                   'width': a['Largura (m)_3'] if a['Largura (m)_3'] is not None else None,
                   'display_amount': True if a['Quantidade (un)_3'] is not None else False,
                   'amount': a['Quantidade (un)_3'] if a['Quantidade (un)_3'] is not None else None                
                  } if a['Identificação do local_3'] is not None else None,
                 {'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_4']) if a['Identificação do local_4'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_4']) if a['Constatação_4'] is not None else None,
                   'place_identification': get_inner_value(array_name, 'Constatação (detalhe)', occ, a['Constatação (detalhe)_4']) if a['Constatação (detalhe)_4'] is not None else None,
                   'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_4']) if a['Irregularidades_4'] is not None else None,
                   'other_description': a['Outros - Descrição_4'] if a['Outros - Descrição_4'] is not None else None,
                   'display_amount': True if a['Quantidade (un)_4'] is not None else False,
                   'amount': a['Quantidade (un)_4'] if a['Quantidade (un)_4'] is not None else None  
                  } if a['Identificação do local_4'] is not None else None,
                 {'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_5']) if a['Identificação do local_5'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_5']) if a['Constatação_5'] is not None else None,
                   'place_identification': get_inner_value(array_name, 'Constatação (detalhe)', occ, a['Constatação (detalhe)_5']) if a['Constatação (detalhe)_5'] is not None else None,
                   'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_5']) if a['Irregularidades_5'] is not None else None,
                   'other_description': a['Outros - Descrição_5'] if a['Outros - Descrição_5'] is not None else None,
                   'display_amount': True if a['Quantidade (un)_5'] is not None else False,
                   'amount': a['Quantidade (un)_5'] if a['Quantidade (un)_5'] is not None else None             
                  } if a['Identificação do local_5'] is not None else None,
                  {'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_6']) if a['Identificação do local_6'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_6']) if a['Constatação_6'] is not None else None,
                   'place_identification': get_inner_value(array_name, 'Constatação (detalhe)', occ, a['Constatação (detalhe)_6']) if a['Constatação (detalhe)_6'] is not None else None,
                   'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_6']) if a['Irregularidades_6'] is not None else None,
                   'other_description': a['Outros - Descrição_6'] if a['Outros - Descrição_6'] is not None else None,
#                    'display_amount': True if a['Quantidade (un)_6'] is not None else False,
#                    'amount': a['Quantidade (un)_6'] if a['Quantidade (un)_6'] is not None else None             
                  } if a['Identificação do local_6'] is not None else None,
                  {'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_7']) if a['Identificação do local_7'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_7']) if a.get('Constatação_7') is not None else None,
                   'place_identification': get_inner_value(array_name, 'Constatação (detalhe)', occ, a['Constatação (detalhe)_7']) if a['Constatação (detalhe)_7'] is not None else None,
                   'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_7']) if a['Irregularidades_7'] is not None else None,
                   'other_description': a['Outros - Descrição_7'] if a['Outros - Descrição_7'] is not None else None,
#                    'display_amount': True if a['Quantidade (un)_7'] is not None else False,
#                    'amount': a['Quantidade (un)_7'] if a['Quantidade (un)_7'] is not None else None             
                  } if a['Identificação do local_7'] is not None else None,
                  {'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_8']) if a['Identificação do local_8'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_8']) if a.get('Constatação_8') is not None else None,
                   'place_identification': get_inner_value(array_name, 'Constatação (detalhe)', occ, a['Constatação (detalhe)_8']) if a['Constatação (detalhe)_8'] is not None else None,
                   'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_8']) if a['Irregularidades_8'] is not None else None,
                   'other_description': a['Outros - Descrição_8'] if a['Outros - Descrição_8'] is not None else None,
#                    'display_amount': True if a['Quantidade (un)_8'] is not None else False,
#                    'amount': a['Quantidade (un)_8'] if a['Quantidade (un)_8'] is not None else None             
                  } if a['Identificação do local_8'] is not None else None]))

  0%|          | 0/1025 [00:00<?, ?it/s]

KeyError: 'Píer'

In [27]:
for a in tqdm(values):
    if a['Data do registro'] is None or a['Data do registro'] == 'None':
        continue
    if isinstance(a['Data do registro'],str):
        a['Data do registro']=a['Data do registro'].strip()
    if isinstance(a['Data do registro'],datetime):
        a['Data do registro']=a['Data do registro'].strftime('%Y-%m-%d')
    if '00:00' in a['Data do registro']:
        a['Data do registro'] = a['Data do registro'].split(' ')[0]

    try:
        a['Data do registro'] = datetime.strptime(a['Data do registro'].strip(), '%Y-%m-%d')
    except:
        a['Data do registro'] = datetime.strptime(a['Data do registro'].strip(), '%d/%m/%Y')

  0%|          | 0/1025 [00:00<?, ?it/s]

ValueError: unconverted data remains:  /
08/06/2020

In [372]:
values[394]['Município'] = 'Rifaina/SP'

In [373]:
for a in tqdm(values[394:]):
    
    
    
    try:
        """Geometria"""
        properties=[]
        point=transform_coordinates(a['Coordenada X'], a['Coordenada Y']) if a['Coordenada X'] is not None else None
        geometry=GeometryCollection()
        geometry.append(point) if a['Coordenada X'] is not None else None
        properties.append({})
#         print(a['obs'])
        """Registro"""
        record=OccurrenceRecord(company=company,
#                                 firm=engie_firm,
                                firm=firm,
                                status=ServiceOrderActionStatus.objects.get(name=a['Status'].replace('elaboração','Elaboração'), companies=company),
                                approval_step=ApprovalStep.objects.get(approval_flow__company=company, name=a['Status'].replace('elaboração','Elaboração')),
#                                 datetime=a['Data do registro'].strftime('%Y-%m-%dT%H:%M:00-03:00'),
                                datetime=a['Data do registro'] if a['Data do registro'] is not None else None,
                                uf_code=get_uf(a['Município'].split('/')[1]),
                                city=City.objects.get(name=a['Município'].split('/')[0]),
                                occurrence_type=occ,
                                created_by=user,
                                geometry=geometry,
                                properties=properties,
                                origin="5",
                                involved_parts=get_involved_parts(a),
                                form_data={
                                    'irregularities': a['irreg'] if 'irreg' in a and a['irreg'] else None,
                                    'agreement_type': True if a['Houve acordo verbal?'] is not None and a['Houve acordo verbal?'].lower() == 'sim' else False,
                                    'deadline': a['Prazo de cumprimento'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Prazo de cumprimento'] is not None else None,
                                    'agreement': a['Acordo realizado com'] if a['Acordo realizado com'] is not None else None,
                                    'affected_area': a['Área afetada (m2)_1'] if a['Área afetada (m2)_1'] is not None else None,
                                    'observation': a['obs'] if 'obs' in a and a['obs'] else None
                                }
                               )
        if not record.form_data['observation']:
            del record.form_data['observation']
            
        if not record.form_data['irregularities']:
            del record.form_data['irregularities']
        
        record.save()        
    
        """Serviço"""        
        service_order=ServiceOrder(company=company, description=(a['Descrição do serviço'] or "Teste"),
                                   kind="LAND",process_type="1", created_by=user)        
        service_order.save()
        
        record.service_orders.add(service_order)
        service_order.responsibles.add(engie_user)
#         service_order.responsibles.add(user)

        """Entregas"""
        for index, item in enumerate(get_action_list(actions, a['Entrega Atual'])):
            
            service_order_action=ServiceOrderAction(name=item['name'],
                                                    estimated_end_date=date.today()+timedelta(weeks=item['timedelta']['weeks']),
                                                    created_by=user,
                                                    service_order=service_order)            
            service_order_action.save()
            
            """Tarefa"""
            if index == 0 and a['Tarefa atual']:
                procedure=Procedure(action=service_order_action,
#                                     responsible=engie_user,
                                    responsible=user,
                                    firm=firm,
#                                     firm=engie_firm
                                    to_do=a['Tarefa atual'],
                                    created_by=user,
                                    deadline=datetime.strptime('11/11/2023 00:00', "%d/%m/%Y %H:%M")
                                   )
                

                procedure.save()
    except Exception as e:
        print(f'ERRO - {e}{record.form_datas}')

  0%|          | 0/1 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2016-09-22 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2016-09-22 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ServiceOrder.opened_at received a naive datetime (2023-10-28 21:19:00.465925) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427:

Procedure matching query does not exist.


WARNING	Property: Unknown Property name. [1:1: word-wrap]


'NoneType' object has no attribute 'is_final'


In [10]:
engie_user=User.objects.get(username='lucas.reis')

In [30]:
Procedure.objects.filter(firm__company=company,created_by=user,responsible=user).count()

991

In [21]:
teste=get_action_list(actions, values[0]['Entrega Atual'])
teste

[{'name': 'Identificar a titularidade do imóvel', 'timedelta': {'weeks': 2}},
 {'name': 'Confirmar o infrator', 'timedelta': {'weeks': 3}},
 {'name': 'Esclarecer o limite da ocupação', 'timedelta': {'weeks': 4}},
 {'name': 'Realizar acordo verbal', 'timedelta': {'weeks': 5}},
 {'name': 'Atestar o cumprimento do acordo verbal', 'timedelta': {'weeks': 6}},
 {'name': 'Notificar o infrator extrajudicialmente',
  'timedelta': {'weeks': 7}},
 {'name': 'Atestar o cumprimento da NE', 'timedelta': {'weeks': 8}},
 {'name': 'Ajuizar ação possessória', 'timedelta': {'weeks': 9}},
 {'name': 'Atender as providências judiciais', 'timedelta': {'weeks': 10}},
 {'name': 'Atestar a desocupação', 'timedelta': {'weeks': 11}},
 {'name': 'Ajuizar ação de execução', 'timedelta': {'weeks': 12}},
 {'name': 'Atestar o cumprimento do mandado', 'timedelta': {'weeks': 14}}]

In [27]:
for x in ServiceOrder.objects.filter(number__in=['UHMI-OS-2023.0015',
                                                 'UHMI-OS-2023.0016',
                                                 'UHMI-OS-2023.0017',
                                                 'UHMI-OS-2023.0018',
                                                'UHMI-OS-2023.0019',
                                                'UHMI-OS-2023.0020',
                                                'UHMI-OS-2023.0021',
                                                'UHMI-OS-2023.0022',
                                                'UHMI-OS-2023.0023']):
    for index, item in enumerate(get_action_list(actions, ServiceOrderAction.objects.get(service_order=x).name)):

        service_order_action=ServiceOrderAction(name=item['name'],
                                                estimated_end_date=date.today()+timedelta(weeks=item['timedelta']['weeks']),
                                                created_by=user,
                                                service_order=x)            
        service_order_action.save()

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1369: RuntimeWarning: DateTimeField ServiceOrderAction.estimated_end_date received a naive datetime (2024-01-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1369: RuntimeWarning: DateTimeField HistoricalServiceOrderAction.estimated_end_date received a naive datetime (2024-01-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1369: RuntimeWarning: DateTimeField ServiceOrderAction.estimated_end_date received a naive datetime (2024-01-08 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db

In [25]:
for x in ServiceOrder.objects.filter(number__in=['UHMI-OS-2023.0015',
                                                 'UHMI-OS-2023.0016',
                                                 'UHMI-OS-2023.0017',
                                                 'UHMI-OS-2023.0018',
                                                'UHMI-OS-2023.0019',
                                                'UHMI-OS-2023.0020',
                                                'UHMI-OS-2023.0021',
                                                'UHMI-OS-2023.0022',
                                                'UHMI-OS-2023.0023']):
    for index, item in enumerate(get_action_list(actions, ServiceOrderAction.objects.get(service_order=x).name)):
        service_order_action=ServiceOrderAction(name=item['name'],
                                                estimated_end_date=date.today()+timedelta(weeks=item['timedelta']['weeks']),
                                                created_by=user,
                                                service_order=service_order)            
        service_order_action.save()

Identificar a titularidade do imóvel
Confirmar o infrator
Esclarecer o limite da ocupação
Realizar acordo verbal
Atestar o cumprimento do acordo verbal
Notificar o infrator extrajudicialmente
Atestar o cumprimento da NE
Ajuizar ação possessória
Atender as providências judiciais
Atestar a desocupação
Ajuizar ação de execução
Atestar o cumprimento do mandado
Identificar a titularidade do imóvel
Confirmar o infrator
Esclarecer o limite da ocupação
Realizar acordo verbal
Atestar o cumprimento do acordo verbal
Notificar o infrator extrajudicialmente
Atestar o cumprimento da NE
Ajuizar ação possessória
Atender as providências judiciais
Atestar a desocupação
Ajuizar ação de execução
Atestar o cumprimento do mandado
Identificar a titularidade do imóvel
Confirmar o infrator
Esclarecer o limite da ocupação
Realizar acordo verbal
Atestar o cumprimento do acordo verbal
Notificar o infrator extrajudicialmente
Atestar o cumprimento da NE
Ajuizar ação possessória
Atender as providências judiciais
Ate

In [24]:
ServiceOrderAction.objects.get(service_order=x).name

'Identificar a titularidade do imóvel'

In [34]:
properties.append({"label":a['N. da Invasão']})

7783921

In [26]:
rep_edit=[]
erros=[]
for a in tqdm(values):
    
    """Geometria"""
    properties=[]
    if a['Coordenada X'] is not None and '23' in str(a['Fuso']).strip() :
        point=transform_coordinates(a['Coordenada X'], a['Coordenada Y'],srid=31983)
    else:
        point=transform_coordinates(a['Coordenada X'], a['Coordenada Y']) if a['Coordenada X'] is not None else None
#     point=transform_coordinates(a['Coordenada X'], a['Coordenada Y']) if a['Coordenada X'] is not None else None
    geometry=GeometryCollection()
    geometry.append(point) if a['Coordenada X'] is not None else None
#     properties.append({})
    try:
        record=OccurrenceRecord.objects.get(company=company,
                                    firm=firm,
                                    status=ServiceOrderActionStatus.objects.get(name=a['Status'].replace('elaboração','Elaboração'), companies=company),
                                    approval_step=ApprovalStep.objects.get(approval_flow__company=company, name=a['Status'].replace('elaboração','Elaboração')),
                                    datetime=a['Data do registro'] if a['Data do registro'] is not None else None,
                                    uf_code=get_uf(a['Município'].split('/')[1]),
                                    city=City.objects.get(name=a['Município'].split('/')[0]),
                                    occurrence_type=occ,
                                    created_by=user,
                                    geometry=geometry,
#                                     properties=properties,
                                    origin="5")
        record.properties=[{"label":a['N. da Invasão']}]
        
        rep_edit.append(a)
    except Exception as e:
        erros.append(a)
        print(e)

  0%|          | 0/1025 [00:00<?, ?it/s]

get() returned more than one OccurrenceRecord -- it returned 10!
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matchi

OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not

OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not

OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not

OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.


In [25]:
len(rep_edit),len(erros)

(332, 693)

In [85]:
for a in tqdm(values):

    try:
        record=OccurrenceRecord.objects.get(company=company,
                                    firm=firm,
                                    status=ServiceOrderActionStatus.objects.get(name=a['Status'].replace('elaboração','Elaboração'), companies=company),
                                    approval_step=ApprovalStep.objects.get(approval_flow__company=company, name=a['Status'].replace('elaboração','Elaboração')),
                                    datetime=a['Data do registro'] if a['Data do registro'] is not None else None,
                                    uf_code=get_uf(a['Município'].split('/')[1]),
                                    city=City.objects.get(name=a['Município'].split('/')[0]),
                                    occurrence_type=occ,
                                    created_by=user,
                                    created_at__gte='2023-10-28',
                                    origin="5")

        record.properties=[{"label":a['N. da Invasão']}]
        record.save()
    except Exception as e:
        erros.append(a)
        continue
    

  0%|          | 0/395 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [84]:
len(erros),erros[0]

(342,
 {'N. da Invasão': 'UHJA-086E',
  'Região': 'Orla',
  'Fuso': 23,
  'Coordenada X': 246661,
  'Coordenada Y': 7777363,
  'Município': 'Rifaina/SP',
  'Status': 'Registro em elaboração',
  'Equipe': 'LR Ambiental',
  'Data do registro': datetime.datetime(2005, 6, 22, 0, 0),
  'Classe': 'Uso e Ocupação Irregular',
  'Identificação do local_1': 'Área de inundação',
  'Identificação do local_2': None,
  'Identificação do local_3': None,
  'Identificação do local_4': None,
  'Identificação do local_5': None,
  'Identificação do local_6': None,
  'Identificação do local_7': None,
  'Identificação do local_8': None,
  'Constatação_1': 'Ocupação/utilização não autorizada',
  'Constatação_2': None,
  'Constatação_3': None,
  'Constatação_4': None,
  'Constatação_5': None,
  'Constatação_6': None,
  'Constatação_7': None,
  'Constatação_8': None,
  'Constatação (detalhe)_1': 'Muro',
  'Constatação (detalhe)_2': None,
  'Constatação (detalhe)_3': None,
  'Constatação (detalhe)_4': None,
  '

In [76]:
ocr=OccurrenceRecord.objects.get(uuid='79c8a660-50ca-4ff9-bdcf-56cf3621f0ba')

In [78]:
ocr.properties=[ocr.properties[1]]

In [79]:
ocr.save()